# DEL 3.4 - Valgfrie felt og `Field`



In [ ]:
# Importer bibliotekene vi trenger
from dataclasses import dataclass
from datetime import datetime
from pydantic import BaseModel, Field


***

## 3.4.1: Valgfrie felt i `pydantic`

For å gjøre et felt valgfritt kan vi gi det en standardverdi i definisjonen:

```python
class Modell(BaseModel):
    paakrevd_felt: str
    valgfritt_felt: None | str = None


# Modell kan initialiseres med og uten verdi for valgfritt_felt:
a = Modell(
    paakrevd_felt="hei"
) # a.valgfritt_felt har verdi None
b = Modell(
    paakrevd_felt="hei", 
    valgfritt_felt="på deg"
) # b.valgfritt felt har verdi "på deg"
```

Dersom du ønsker at et valgfritt felt skal ha en annen standard-verdi enn
`None` kan dette også settes:

```python
class Modell(BaseModel):
    paakrevd_felt: str
    valgfritt_felt: str = "tom"


# Modell kan initialiseres med og uten verdi for valgfritt_felt:
a = Modell(
    paakrevd_felt="hei"
) # a.valgfritt_felt har verdi "tom"
b = Modell(
    paakrevd_felt="hei", 
    valgfritt_felt="på deg"
) # b.valgfritt felt har verdi "på deg"
```


## 3.4.2: `Field` - Ytterligere konfigurasjon av egenskaper / felt 

Pydantic-biblioteket har en funksjon `Field` som gir deg mulighet til å gi
ytterligere egenskaper til et datafelt. Som `ConfigDict` må du huske å importere
`Field` fra `pydantic`. Dette er allerede gjort i første kodecelle med linjen:

In [ ]:
from pydantic import BaseModel, Field

Før musen over `Field` i cellen over, og se litt på parameterne til funksjonen. Du kan se at det er ganske mange muligheter! Vi skal ta for oss noen av de mest nyttige her:

In [ ]:
class TestModell(BaseModel):
    postnummer: str = Field(
        pattern=r'^\d{4}$',                    # Regex mønster for å validere input
        description="Postnummer (4 siffer)"    # Beskrivelse av feltet
    )
    positivt_tall: int = Field(
        default=1,                             # Standardverdi for feltet
        gt=0,                                  # Verdi må være større enn 0
        alias="positivtTall",                  # Alias for feltet i input/output
        description="Et positivt heltall"      # Beskrivelse av feltet
    )
    landkode: str = Field(
        min_length=2,                          # Minimum lengde på strengen
        max_length=3,                          # Maksimum lengde på strengen
        alias="countryCode",                   # Alias for feltet i input/output
        description="Landkode (2-3 bokstaver)" # Beskrivelse av feltet
    )
    prosent: int = Field(
        ge=0,                                  # Verdi må være større enn eller lik 0
        le=100,                                # Verdi må være mindre enn eller lik 100
    )

Over ser vi eksempler på definisjoner av felt som inkluderer __validering__,
__standardverdi__, __alias__ og __beskrivelse__.

Parametrene `pattern`, `gt`, `ge`, `le`, `min_length` og `max_length` er eksempler på verdier som kan settes for å gi ytterligere
validering av inn-data. 

Parameteret `default` angir standardverdi, og fungerer likt som vist over med
`valgfritt_felt: str = "tom"`. 

Parameteret `alias` angir et alternativt navn for datafeltet, slik at du kan
bruke et annet navn for feltet enn feltnavnet i kildedata. For eksempel kan du
omdøpe et navn fra camelCase til snake_case, da camelCase er vanlig i json, mens
det er foretrukket å bruke snake_case i python (se definisjon av `positivt_tall`
over).

In [ ]:
## Demo av TestModell
gyldig_data_1 = {
    "postnummer": "1234",
    "positivtTall": 10,  # Merk: bruker alias, ikke navnet på egenskapen
    "countryCode": "NO", # Merk: bruker alias, ikke navnet på egenskapen
    "prosent": 85,
}

gyldig_data_2 = {
    "postnummer": "5678",
    # Merk: positivt_tall bruker standardverdi, så det kan utelates
    "countryCode": "SE",
    "prosent": 90,
}

ugyldig_data_1 = {
    "postnummer": "12A4",    # Inneholder bokstav
    "positivtTall": -5,      # Negativt tall
    "countryCode": "NORGE",  # For langt
    "prosent": 110,          # Mer enn 100
}

ugyldig_data_2 = {
    "postnummer": "123",     # For kort
    "positivtTall": 5.1,     # Flyttall i stedet for heltall
    "landkode": "D",         # Bruker feil navn (skal bruke alias)
    "prosent": -10,          # Negativt tall
}

gyldig_1 = TestModell(**gyldig_data_1)
print("Gyldig data 1:")
print(gyldig_1)

gyldig_2 = TestModell(**gyldig_data_2)
print("\nGyldig data 2:")
print(gyldig_2)

try:
    ugyldig_1 = TestModell(**ugyldig_data_1)
except Exception as e:
    print("\nFeil ved ugyldig data 1:")
    print(e)

try:
    ugyldig_2 = TestModell(**ugyldig_data_2)
except Exception as e:
    print("\nFeil ved ugyldig data 2:")
    print(e)

print("\n--- Feltbeskrivelser i TestModell ---")
for field_name, field in TestModell.model_fields.items():
    print(f"{field_name}: {field.description}")

## VEDLEGG: Objekter som standardverdier

> Dette er ekstra lesing som tar opp et viktig tema i programmering.
> Det omhandler variabler, minnehåndtering og objektreferanser. Dette
> håndteres litt ulikt i forskjellige programmeringsspråk, men det er 
> viktig å ha en viss forståelse om hvordan språket man skriver håndterer
> dette. Ved feil bruk / forståelse av objektreferanser, sniker det seg
> fort inn feil som kan være litt vanskelige å oppdage og forstå.

Som vi har vært inne på tidligere er de basale datatypene, som  `int`, `str`,
`float` og `bool`, _immutable_. Det betyr at verdien ikke kan endres etter den
er satt. Med andre ord er minnelokasjonen i RAM som inneholder verdien låst, og
kan ikke endres. Det som skjer når du for eksempel legger sammen to `int`, er at
et nytt `int`-objekt opprettes, som får tildelt en ny minneadresse. Dette gjelder
også for `tuple`, som også er _immutable_. Vi kan se dette med å bruke funksjonen
`id` som skriver ut minneadressen til en variabel. Kjør cellen under som 
demonstrerer dette, og skriver ut forklaringer om hva som skjer:

In [ ]:
print("\n### Demonstrasjon av mutable og immutable typer ###")

print("\nImmutable type (int):")
print("-------------------\n")
tall = 5
tall_kopi = tall
print(f"tall: {tall}, id: {id(tall)}")
print(f"tall_kopi: {tall_kopi}, id: {id(tall_kopi)}")
print("\n***************")
print("* Vi ser at både tall og tall_kopi har samme minneadresse (id),")
print("* dette betyr at de peker til samme objekt i minnet.")
print("***************")

print("\n-------------------")
print("Nå endrer vi verdien av tall (tall += 10):")
tall += 10 # Vi legger til 10 til verdien av tall
print("Etter endring av tall:")
print("-------------------")
print(f"tall: {tall}, id: {id(tall)}")
print(f"tall_kopi: {tall_kopi}, id: {id(tall_kopi)}")

print("\n***************")
print("* Vi ser at etter endring av tall, har den fått en ny minneadresse (id),")
print("* mens tall_kopi fortsatt peker til det opprinnelige objektet.")
print("* Altså endres ikke verdien lagret i den originale minnelokasjonen til tall seg!")
print("***************")


print("\nMutable type (liste):")
print("-------------------\n")
liste = [1, 2, 3]
liste_kopi = liste
print(f"liste: {liste}, id: {id(liste)}")
print(f"liste_kopi: {liste_kopi}, id: {id(liste_kopi)}")
print("\n***************")
print("* Vi ser at både liste og liste_kopi har samme minneadresse (id),")
print("* dette betyr at de peker til samme objekt i minnet.")
print("***************")

print("\n-------------------")
print("Nå endrer vi listen (liste.append(4)):")
liste.append(4)  # Vi legger til et element i listen
print("Etter endring av liste:")
print("-------------------")
print(f"liste: {liste}, id: {id(liste)}")
print(f"liste_kopi: {liste_kopi}, id: {id(liste_kopi)}")

print("\n***************")
print("* Vi ser at etter endring av liste, beholdes den samme minneadressen (id).")
print("* Endringen har altså skjedd på samme minnelokasjon,")
print("* og liste peker fortsatt til det opprinnelige objektet.")
print("* Siden endringen skjedde på samme minnelokasjon, har også verdien i liste_kopi endret seg!")
print("***************")


Når vi jobber med _immutable_ objekter, trenger vi ikke å tenke på at verdien
til en variabel kan endres utilsiktet når programmet kjører. Med _mutable_
objekter derimot, kan vi fort komme til skade for å gjøre endringer i verdien
til andre objekter enn det som var hensikten. Dette får også konsekvenser for
standardverdier på parametre, enten det er snakk om funksjonsdefinisjoner eller
definisjoner av klasser. 

La oss først vise dette med en funksjon der parameteret gis en standardverdi
som en tom liste:

In [ ]:
def legg_til_element(element: int, liste: list[int] = []) -> None:
    print(f"verdi og id til parameteret liste ved funksjonskall: {liste}, id: {id(liste)}")
    liste.append(element)
    return liste

def demo_legg_til_element():
    print("Vi kaller legg_til_element første gang med verdi 1, og lagrer resultatet i liste1:")
    liste1 = legg_til_element(1)
    print(f"Etter første kall, liste1: {liste1}")
    print("\nNå kaller vi legg_til_element en gang til med verdi 2, og lagrer resultatet i liste2:")
    liste2 = legg_til_element(2)
    print(f"Etter andre kall, liste2: {liste2}")
    print("Nå legger vi til verdi 3 i liste2 med append:")
    liste2.append(3)
    print(f"Etter append, liste2: {liste2}")

    print("\n***************")
    print("La oss nå sjekke verdi og id til liste1 og liste2:")
    print(f"liste1: {liste1}, id: {id(liste1)}")
    print(f"liste2: {liste2}, id: {id(liste2)}")

demo_legg_til_element()

**Hva skjedde her?**

Kodelinjen som er funksjonshodet
(`def legg_til_element(element: int, liste: list[int] = []) -> None:`) 
initialiseres bare én gang, og standardverdien til parameteret liste vil ha samme 
id / minnelokasjon for alle kall til funksjonen. Dermed bør man alltid unngå å
sette standardverider til _mutable_ objekter, for det kan fort skape trøbbel!

Dette kan enkelt løses med at standardverdien til et parameter settes i
funksjonskroppen i stedet for i funksjonshodet. Da er det også vanlig å gi
parameteret standardverdien `None` i funksjonshodet, som gjør parameteret
valgfritt. Her er en trygg versjon av funksjonen `legg_til_element`. Ved å kjøre
cellen under redefineres `legg_til_element`, og samme test kjøres som i cellen over.
Se på utskriften at vi nå får et annet resultat!

In [ ]:
def legg_til_element(element: int, liste: list[int] | None = None) -> None:
    # Nå settes standardverdien til None i hodet på funksjonen
    # og vi setter den faktiske standardverdien inne i funksjonen:
    if liste is None:
        liste = []

    print(f"verdi og id til parameteret liste ved funksjonskall: {liste}, id: {id(liste)}")
    liste.append(element)
    return liste

demo_legg_til_element()

> Hvis du får feilmelding om at `demo_legg_til_element`, må du kjøre
> den forrige kodecellen (hvor funksjonen defineres) først.

**Hvordan ser dette ut med klasser?**

Under har vi samme utfordring der standardverdi til parameter i `__init__`
funksjonen er en mutable klasse (`list`)


In [ ]:
class Handlekurv:
    def __init__(self, kunde_id: str, produkter: list[str] = []):
        self.kunde_id = kunde_id
        self.produkter = produkter

def print_produkter(handlekurv: Handlekurv):
    print(f"Handlekurv for kunde {handlekurv.kunde_id} inneholder produktene:")
    for produkt in handlekurv.produkter:
        print(f"- {produkt}")

def demo_handlekurv():
    kurv1 = Handlekurv(kunde_id="Kunde1")
    kurv1.produkter.append("Eple")
    print(f"Vi har opprettet en handlekurv for {kurv1.kunde_id} og lagt til ett produkt (Eple).")
    print_produkter(kurv1)

    kurv2 = Handlekurv(kunde_id="Kunde2")
    kurv2.produkter.append("Banan")
    print(f"Vi har opprettet en handlekurv for {kurv2.kunde_id} og lagt til ett produkt (Banan).")
    print_produkter(kurv2)

    print("\n***************")
    print("La oss sjekke id og verdi på de to produktlistene:")
    print(f"Handlekurv 1 (Kunde1) - ID: {id(kurv1.produkter)}, Verdi: {kurv1.produkter}")
    print(f"Handlekurv 2 (Kunde2) - ID: {id(kurv2.produkter)}, Verdi: {kurv2.produkter}")

demo_handlekurv()

Dette fikses enkelt på samme måte som med funksjoner! Kjør under for å se:

In [ ]:
class Handlekurv:
    def __init__(self, kunde_id: str, produkter: list[str] | None = None):
        self.kunde_id = kunde_id
        self.produkter = produkter if produkter is not None else []

demo_handlekurv()

**Hva med dataklasser?**

Under er en dataklasse-definisjon, som tilsvarer den tidligere klassedefinisjonen. Kjør cellen og se hva som skjer!

In [ ]:
try:
    @dataclass
    class Handlekurv:
        kunde_id: str
        produkter: list[str] = []
except Exception as e:
    print("Feil ved definisjon av dataclass Handlekurv med mutable standardverdi:")
    print(e)

Vi får altså ikke lov til å gjøre dette med dataclass, og får en feilmelding!
Her må default_factory brukes, som er et parameter på funksjonen `field` i
dataclass-biblioteket:

```python
@dataclass
class Handlekurv:
    kunde_id: str
    produkter: list[str] = field(default_factory=list)
```
    
Denne funksjonen ligner veldig på `Field` fra `pydantic`. Mye i `dataclass`
og `pydantic` ligner, og du kan se på `pydantic` som en utvidelse av
`dataclass`, med mer funksjonalitet for datavalidering. 

For å unngå forvirring hopper vi videre fra `dataclass` til `pydantic`!

**Så hvordan blir dette i `pydantic`?** 

La oss redefinere `Handlekurv` og kjøre
samme test, for å se hva som skjer når vi lar en tom liste være init verdi til
en egenskap i en `pydantic`-modell:



In [ ]:
class Handlekurv(BaseModel):
    kunde_id: str
    produkter: list[str] = []

demo_handlekurv()

_Her fikk vi ikke samme feil!_ 

`Pydantic` er faktisk så greie med oss at det gjøres en _deep copy_
av _mutable_ objekter ved initialisering, slik at objektet får kopieres
til nye minnelokasjoner. Derfor er det trygt å sette slike enkle 
standardverdier som en tom liste når vi bruker `pydantic`. Vi har likevel
et `default_factory`-parameter i `Field` som også kan brukes! 

**Hva er en såkalt `default_factory`?**

Dette er en funksjon som kjøres hver gang et nytt objekt skal initialiseres
med en standardverdi, for å opprette en ny uavhengig variabel i minnet.

Vi kan bruke dette parameteret slik:

In [ ]:
class Handlekurv(BaseModel):
    kunde_id: str
    produkter: list[str] = Field(default_factory=list)

demo_handlekurv()

Siden list også er et klassenavn ser det ikke ut som en funksjon, men
`default_factory=list` er det samme som `default_factory=lambda: list()`, så det
kjøres altså en funksjon som oppretter en ny, tom liste med `list()`

Selv om vi ikke trenger `default_factory` for trygg init av _mutable_ objekter, 
kan vi gjøre andre artige ting med `default_factory`. Vi kan for eksempel legge
til en egenskap ved `Handlekurv` som gir oss timestamp på når den ble opprettet,
der øyeblikket et nytt objekt lages blir satt som standardverdi:


In [ ]:
class Handlekurv(BaseModel):
    kunde_id: str
    opprettet: datetime = Field(default_factory=datetime.now)
    produkter: list[str] = []

handlekurv1 = Handlekurv(kunde_id="Kunde1")
print(f"Handlekurv 1 ble opprettet {handlekurv1.opprettet.strftime('%d.%m.%Y %H:%M:%S')}")

> Sjekk ut ⬆️
>
> Kjør cellen over flere ganger for å se at verdien endrer seg

Et annet eksempel er en funksjon som genererer kunde-id når nytt
objekt opprettes:

In [ ]:
def generer_kunde_id() -> str:
    """Genererer kunde-ID basert på tidsstempel."""
    return f"KUNDE-{str(datetime.now().timestamp()).replace('.', '')[-5:]}"
class Handlekurv(BaseModel):
    kunde_id: str = Field(default_factory=generer_kunde_id)
    opprettet: datetime = Field(default_factory=datetime.now)
    produkter: list[str] = []

handlekurv1 = Handlekurv()
print(
    "Handlekurv 1 ble opprettet for kunde", 
    f"med ID {handlekurv1.kunde_id} den",
    handlekurv1.opprettet.strftime('%d.%m.%Y %H:%M:%S')
)

> Sjekk ut ⬆️
>
> Kjør cellen over flere ganger for å se at verdien endrer seg